In [1]:
import pandas as pd
import datetime
from datetime import timedelta

# Load the staff data
staff_data = pd.read_excel('staff.xlsx')

# Handle missing values in the 'Type' column (example: fill with 'Unknown')
staff_data['Type'] = staff_data['Type'].fillna('Unknown')

# Group and aggregate data for deployment plan calculations
grouped_data = staff_data.groupby(['Project', 'Job', 'Type']).agg(
    Peak_Number=('Job', 'count'),
    Average_Start_Month=('Month', 'min'),
    Average_End_Month=('Month', 'max')
).reset_index()

# Convert 'Month' columns to datetime
grouped_data['Average_Start_Month'] = pd.to_datetime(grouped_data['Average_Start_Month'])
grouped_data['Average_End_Month'] = pd.to_datetime(grouped_data['Average_End_Month'])

# Define the function to create the deployment plan for each job title
def create_deployment_plan_by_job_title(project_type, duration_months):

    # Filter data based on project type
    filtered_data = grouped_data[grouped_data['Type'] == project_type]

    # Create a list to store deployment plans for each job title
    deployment_plans = []

    for job_title, job_data in filtered_data.groupby('Job'):

        # Calculate average number of jobs for the job title
        average_job_count = job_data['Peak_Number'].mean()  # Define `average_job_count` here
        
        # Calculate average start and end months within the dataset
        avg_start_month = job_data['Average_Start_Month'].mean().month
        avg_end_month = job_data['Average_End_Month'].mean().month

        # Adjust start and end months based on user input and dataset averages
        start_month = min(avg_start_month, 1 + duration_months)
        end_month = min(avg_end_month, start_month + duration_months - 1)

        # Calculate actual start and end dates within the specified duration
        start_date = datetime.datetime(2024, 1, 1) + timedelta(days=31 * (start_month - 1))
        end_date = start_date + timedelta(days=31 * (end_month - start_month + 1))

        # Create the deployment plan dictionary for the job title
        deployment_plan = {
            'Project_Type': project_type,
            'Job_Title': job_title,
            'Number_of_Jobs': int(average_job_count),
            'Start_Month': start_month,
            'End_Month': end_month,
            'Start_Date': start_date.strftime("%Y-%m-%d"),
            'End_Date': end_date.strftime("%Y-%m-%d"),
        }

        deployment_plans.append(deployment_plan)

    return deployment_plans

# Get user input
project_type = input("Enter the project type: ")
duration_months = int(input("Enter the project duration in months: "))

# Create and print the deployment plans for each job title
deployment_plans = create_deployment_plan_by_job_title(project_type, duration_months)
print("Deployment Plans for", project_type, "Project by Job Title:")
for plan in deployment_plans:
    print(plan)

# Create the Excel writer object before writing to the file
writer = pd.ExcelWriter('deployment_plans.xlsx')

# Create a single DataFrame to hold all deployment plans
all_plans_df = pd.DataFrame(deployment_plans)

# Write the DataFrame to a single worksheet in the Excel file
all_plans_df.to_excel(writer, sheet_name="Deployment Plans", index=False)

# Close the writer to save the Excel file
writer.close()

print("Deployment plans saved to deployment_plans.xlsx")

Deployment Plans for Bridges Project by Job Title:
{'Project_Type': 'Bridges', 'Job_Title': 'Assistant Store -Keeper', 'Number_of_Jobs': 2, 'Start_Month': 5, 'End_Month': 6, 'Start_Date': '2024-05-04', 'End_Date': '2024-07-05'}
{'Project_Type': 'Bridges', 'Job_Title': 'Assistant Store Keeper', 'Number_of_Jobs': 15, 'Start_Month': 9, 'End_Month': 4, 'Start_Date': '2024-09-05', 'End_Date': '2024-05-04'}
{'Project_Type': 'Bridges', 'Job_Title': 'Blacksmith Supervisor', 'Number_of_Jobs': 3, 'Start_Month': 5, 'End_Month': 6, 'Start_Date': '2024-05-04', 'End_Date': '2024-07-05'}
{'Project_Type': 'Bridges', 'Job_Title': 'Bridges Projects Manager', 'Number_of_Jobs': 1, 'Start_Month': 1, 'End_Month': 1, 'Start_Date': '2024-01-01', 'End_Date': '2024-02-01'}
{'Project_Type': 'Bridges', 'Job_Title': 'Carpenter Supervisor', 'Number_of_Jobs': 4, 'Start_Month': 11, 'End_Month': 3, 'Start_Date': '2024-11-06', 'End_Date': '2024-04-03'}
{'Project_Type': 'Bridges', 'Job_Title': 'Chief Surveyor', 'Number_